In [1]:
from unityagents import UnityEnvironment
env_file = "Tennis_Windows_x86_64\Tennis.exe" #'Tennis_Linux/Tennis.x86_64'#
env = UnityEnvironment(file_name=env_file, no_graphics = True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [2]:
import random
import sys
import torch
import numpy as np
from collections import deque
import progressbar as pb
import matplotlib.pyplot as plt
from utils import OUprocess, Reply_buffer
from Tennis_agent import SelfPlay_Agent

using device:  cpu
NO grad. clipping used.


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('states shape: ',states.shape)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print('The state for the second agent looks like:', states[1])

Number of agents: 2
Size of each action: 2
states shape:  (2, 24)
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]
The state for the second agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.4669857  -1.5
  0.          0.         -6.83172083  6.          0.          0.        ]


In [5]:
def train_agent(env, nr_episodes, num_agents, seed): 
    
    max_buffer = int(1.e+6)
    
    noise_decay = 1#0.999
    noise_lvl = 0.2
    
    batch_size = 256
    update_steps = 5
    GD_steps = 4
    lra=1.e-3
    lrc=1.e-3
    tau=1
    discount=0.999
    
    print('### ===================================== seed {} ======================================= ###'.format(seed))
    print('lra {}, lrc {}, noise_lvl {}, noise_decay {}, batch_size {}, update_steps {}, GD_steps {}, tau {}, gamma {}'.format(
        lra, lrc, noise_lvl,noise_decay,batch_size,update_steps,GD_steps,tau,discount))
    
    buffer = Reply_buffer(max_buffer,seed=seed)
    ou = OUprocess(num_agents, action_size, sigma=noise_lvl, seed=seed)
    agent = SelfPlay_Agent(state_size, action_size, num_agents, tau=tau, discount=discount, lr_act=lra, lr_crit=lrc, seed=seed)
    print(agent.actor)
    print(agent.critic)
    widget = ['training loop: ', pb.Percentage(), ' ', pb.Bar()]
    timer = pb.ProgressBar(widgets=widget, maxval=nr_episodes).start()
    ACloss = []
    l_exists = False
    max_avg = np.inf
    e_score = np.zeros(nr_episodes)
    rolling_avg = np.zeros(nr_episodes)
    rolling_window = deque(maxlen=100)
    t=0
    for i in range(nr_episodes):                                         # play game for nr_episodes episodes
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        ou.reset()
    #     ouB.reset()
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        while True:
            t+=1
            # select action
            agent.actor.eval()
            noise = ou.noise()
            actA  = agent.act(states[np.newaxis,0]).data.cpu().numpy() + noise[0]
            actB = agent.act(states[np.newaxis,1]).data.cpu().numpy() + noise[1]
            actions = np.clip([actA[0], actB[0]],-1,1) # select an action (for each agent)
            # execute and observe
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            # store in replay buffer
            samples = [states, np.asarray(actions), np.asarray(rewards), next_states, dones]
            buffer.store(samples)
            # roll over states to next time step
            states = next_states                               
            # keep track of scores
            scores += env_info.rewards                         # update the score (for each agent)
            # Update agents
            if ((t-1) % update_steps == 0) and len(buffer.buffer) > batch_size:
                l_exists = True
                agent.actor.train()
                for ep in range(GD_steps):
                    minibatch = buffer.get_batch(batch_size)  # get batch of size (batch_size x 5) where the 2nd dim corresponds to (s,a,r,s',done)
                    agent_i = np.random.choice(num_agents,len(minibatch))
                    closs = agent.update_critic(minibatch,agent_i)
                    aloss = agent.update_actor(minibatch,agent_i)

            # Update target network
            if agent.tau == 1 and t% int(2*update_steps) == 0:
                agent.target_update()
            else:
                agent.target_update()
            # exit loop if episode finished
            if np.any(dones): 
                break

        if l_exists:
            ACloss.append([aloss,closs])

        ou.sigma *= noise_decay

        e_score[i] = np.max(scores)
        rolling_window.append(e_score[i])
        rolling_avg[i] = np.mean(np.array(rolling_window))
        if rolling_avg[i] > 0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i, rolling_avg[i]))
            # Save actor and critic
            torch.save(agent.actor.state_dict(), 'Tennis.player')
            torch.save(agent.critic.state_dict(), 'Tennis.critic')
            max_avg = rolling_avg[i]
            print('Networks saved')

        if (i+1) % 20 == 0:
            print('Total score (averaged over agents) episode {}: {}'.format(i+1,np.mean(scores)))
            print('Score (max over agents) from episode {}: {}'.format(i+1, e_score[i]))
            print('Rolling average score: {}'.format(rolling_avg[i]))
    #         print('Noise level: ',ouA.sigma,ouB.sigma)
            if l_exists:
                print("Actor loss: {} | Critic loss: {}".format(*ACloss[-1]))
            if rolling_avg[i]>max_avg:
                # Save best actor and critic
                torch.save(agent.actor.state_dict(), 'Tennis.player')
                torch.save(agent.critic.state_dict(), 'Tennis.critic')
                max_avg = rolling_avg[i]
                print('Better network saved!')
        # update progress widget bar
        timer.update(i+1)

    print('Max score: ', rolling_avg.max())
    timer.finish()
    
    print('### ================================================================================== ###')
    ACloss = np.asarray(ACloss)
    return ACloss, rolling_avg, e_score, agent

Total score (averaged over agents) episode 20: -0.004999999888241291
Score (max over agents) from episode 20: 0.0
Rolling average score: 0.004500000085681677


Total score (averaged over agents) episode 40: -0.004999999888241291
Score (max over agents) from episode 40: 0.0
Rolling average score: 0.009750000154599547
Actor loss: 0.08149810135364532 | Critic loss: 1.738916034810245e-05


Total score (averaged over agents) episode 60: -0.004999999888241291
Score (max over agents) from episode 60: 0.0
Rolling average score: 0.0081666667945683
Actor loss: 0.07704493403434753 | Critic loss: 3.185501554980874e-05


Total score (averaged over agents) episode 80: -0.004999999888241291
Score (max over agents) from episode 80: 0.0
Rolling average score: 0.008625000133179128
Actor loss: 0.07537087798118591 | Critic loss: 2.8992890293011442e-05


Total score (averaged over agents) episode 100: -0.004999999888241291
Score (max over agents) from episode 100: 0.0
Rolling average score: 0.00980000015348196
Actor loss: 0.07332167029380798 | Critic loss: 9.574408977641724e-06


...

Total score (averaged over agents) episode 500: -0.004999999888241291
Score (max over agents) from episode 500: 0.0
Rolling average score: 0.004700000081211329
Actor loss: 0.02801002934575081 | Critic loss: 3.132274287054315e-05


Total score (averaged over agents) episode 520: -0.004999999888241291
Score (max over agents) from episode 520: 0.0
Rolling average score: 0.003800000064074993
Actor loss: 0.028722865507006645 | Critic loss: 2.0397295884322375e-05


Total score (averaged over agents) episode 540: -0.004999999888241291
Score (max over agents) from episode 540: 0.0
Rolling average score: 0.0029000000469386576
Actor loss: 0.026890160515904427 | Critic loss: 1.8102080503012985e-05


Total score (averaged over agents) episode 560: -0.004999999888241291
Score (max over agents) from episode 560: 0.0
Rolling average score: 0.0029000000469386576
Actor loss: 0.02311968430876732 | Critic loss: 2.18865698116133e-05


KeyboardInterrupt: 

In [ ]:
nr_episodes = 20#1400
for i in range(10):
    seed = np.random.randint(10000000)
    ACloss, rolling_avg, e_score, agent = train_agent(env, nr_episodes, num_agents, seed)
    if rolling_avg.max() > 0.5:
        print('Tennis solved!')
        break

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(e_score)
plt.plot(rolling_avg,'r')
plt.plot(0.5*np.ones_like(rolling_avg),'k')
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(ACloss[:,0])
plt.subplot(122)
plt.plot(ACloss[:,1])

lra 0.0005, lrc 0.0005, noise_lvl 0.2, noise_decay 1, batch_size 128, update_steps 1, GD_steps 2 => 0.25

In [ ]:
T_max = 2000
agent.actor.eval()
for i in range(5):                                         # play game for nr_episodes episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    ou.reset()
#     ouB.reset()
    while True:#t<T_max:
        t+=1
        # select action
        noise = ou.noise()
        actA  = agent.act(states[np.newaxis,0]).data.cpu().numpy() + noise[0]
        actB = agent.act(states[np.newaxis,1]).data.cpu().numpy() + noise[1]
        actions = np.clip([actA[0], actB[0]],-1,1) # select an action (for each agent)
        # execute and observe
        #actions = [np.array([0.1,1]),np.array([0.1,1])]
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        # roll over states to next time step
        states = next_states                               
        # keep track of scores
        scores += env_info.rewards                         # update the score (for each agent)
        # exit loop if episode finished
        if np.any(dones): 
            break
    print('Best player score in episode {}: {}'.format(i,np.max(scores)))

In [ ]:
# env.close()